In [1]:
import yfinance as yf
import pandas as pd
from IPython.display import display
from scipy.stats import zscore
import numpy as np
import torch
import gym
from envs.portfolio_env import Mult_Asset_portEnv
import random

from stable_baselines3 import A2C  
from stable_baselines3.common.vec_env import DummyVecEnv


2025-06-24 22:14:27.501364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750799668.258957   75403 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750799668.468102   75403 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-24 22:14:29.322764: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
assets = ['NVDA', 'AAPL', 'AMZN', 'JPM', 'IBM', 'MSFT', 'TSLA', 'GOOGL', 'META', 'HSBC']
start_date ='2019-01-01'
end_date = '2025-01-01'
 
df = yf.download(tickers=assets, start= start_date , end= end_date, interval='1d')

df 

/tmp/ipykernel_75403/1496721472.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=assets, start= start_date , end= end_date, interval='1d')


[*********************100%***********************]  10 of 10 completed


Price            Close                                                 \
Ticker            AAPL        AMZN       GOOGL       HSBC         IBM   
Date                                                                    
2019-01-02   37.617851   76.956497   52.419621  27.954201   82.193153   
2019-01-03   33.870842   75.014000   50.967827  27.646486   80.552292   
2019-01-04   35.316753   78.769501   53.582150  28.453381   83.698456   
2019-01-07   35.238152   81.475502   53.475288  28.070446   84.290627   
2019-01-08   35.909897   82.829002   53.944977  28.145668   85.489151   
...                ...         ...         ...        ...         ...   
2024-12-24  257.578674  229.050003  195.647552  47.340084  221.451202   
2024-12-26  258.396667  227.050003  195.138763  47.349697  221.924866   
2024-12-27  254.974930  223.750000  192.305450  47.378532  219.842682   
2024-12-30  251.593079  221.300003  190.789032  47.590000  217.346039   
2024-12-31  249.817368  219.389999  188.853622  47.541939  216.931580   

Price                                                                   ...  \
Ticker             JPM        META        MSFT        NVDA        TSLA  ...   
Date                                                                    ...   
2019-01-02   82.408600  134.939697   94.945488    3.377545   20.674667  ...   
2019-01-03   81.237457  131.021194   91.452660    3.173484   20.024000  ...   
2019-01-04   84.232292  137.197342   95.706047    3.376801   21.179333  ...   
2019-01-07   84.290840  137.296783   95.828117    3.555571   22.330667  ...   
2019-01-08   84.131912  141.752319   96.522942    3.467054   22.356667  ...   
...                ...         ...         ...         ...         ...  ...   
2024-12-24  239.589218  606.742920  437.647400  140.197372  462.279999  ...   
2024-12-26  240.409912  602.350220  436.432068  139.907410  454.130005  ...   
2024-12-27  238.462036  598.816101  428.881104  136.987885  431.660004  ...   
2024-12-30  236.632812  590.260254  423.202911  137.467804  417.410004  ...   
2024-12-31  237.018433  584.539795  419.885681  134.268326  403.839996  ...   

Price          Volume                                                   \
Ticker           AAPL       AMZN     GOOGL     HSBC      IBM       JPM   
Date                                                                     
2019-01-02  148158800  159662000  31868000  2565000  4434935  15670900   
2019-01-03  365248800  139512000  41960000  1479400  4546648  16286400   
2019-01-04  234428400  183652000  46022000  3060900  4683779  16935200   
2019-01-07  219111200  159864000  47446000  2212300  3923755  15430700   
2019-01-08  164101200  177628000  35414000  2888300  4982726  13578800   
...               ...        ...       ...      ...      ...       ...   
2024-12-24   23234700   15007500  10403300   309000  1186200   3729100   
2024-12-26   27237100   16146700  12046600   516300  3286500   4451800   
2024-12-27   42355300   27367100  18891400  1088100  1810800   5730200   
2024-12-30   35557500   28321200  14264700  1344500  2095600   5723800   
2024-12-31   39480700   24819700  17466900   768200  2270200   4871000   

Price                                                 
Ticker          META      MSFT       NVDA       TSLA  
Date                                                  
2019-01-02  28146200  35329300  508752000  174879000  
2019-01-03  22717900  42579100  705552000  104478000  
2019-01-04  29002100  44060600  585620000  110911500  
2019-01-07  20089300  35656100  709160000  113268000  
2019-01-08  26263800  31514400  786016000  105127500  
...              ...       ...        ...        ...  
2024-12-24   4726100   7164500  105157000   59551800  
2024-12-26   6081400   8194200  116205600   76366400  
2024-12-27   8084200  18117700  170582600   82666800  
2024-12-30   7025900  13158700  167734700   64941000  
2024-12-31   6019500  13246500  155659200   76825100  

[1510 rows x 50 columns]

In [3]:
sta = df.stack()

df_flt = sta.reset_index()
is_multi_index = isinstance(df_flt.index, pd.MultiIndex)  
is_multi_col= isinstance(df_flt.columns, pd.MultiIndex)

print (df_flt.head())
print (df_flt.info())
print (df_flt.describe())
print (df_flt.dtypes)
print (df_flt.isna().sum())
print(df_flt.index.to_series().min())
print(df_flt.index.to_series().max())
print ('Row', is_multi_index)
print ('col', is_multi_col)

/tmp/ipykernel_75403/3544417225.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  sta = df.stack()


Price       Date Ticker      Close       High        Low       Open     Volume
0     2019-01-02   AAPL  37.617851  37.839387  36.738862  36.896080  148158800
1     2019-01-02   AMZN  76.956497  77.667999  73.046501  73.260002  159662000
2     2019-01-02  GOOGL  52.419621  52.723299  50.958383  51.053811   31868000
3     2019-01-02   HSBC  27.954201  27.974714  27.523399  27.605456    2565000
4     2019-01-02    IBM  82.193153  82.742487  79.681917  79.910209    4434935
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15100 entries, 0 to 15099
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    15100 non-null  datetime64[ns]
 1   Ticker  15100 non-null  object        
 2   Close   15100 non-null  float64       
 3   High    15100 non-null  float64       
 4   Low     15100 non-null  float64       
 5   Open    15100 non-null  float64       
 6   Volume  15100 non-null  int64         
dtypes: datetime64[ns]

In [4]:
# for symbol in assets:
#     df = yf.download(tickers=symbol, start= start_date , end= end_date, interval='1d')
#     sta = df.stack()
#     df_flt = sta.reset_index()
#     df_flt.to_csv(f'/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/{symbol}_daily.CSV')


In [5]:

appl = pd.read_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/AAPL_daily.CSV', parse_dates=["Date"], index_col="Date")
msft = pd.read_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/raw/MSFT_daily.CSV', parse_dates=["Date"], index_col="Date")


df = pd.concat([appl, msft])


is_multi_index = isinstance(df.index, pd.MultiIndex)  
is_multi_col= isinstance(df.columns, pd.MultiIndex)

df = df.drop(columns=['Unnamed: 0']).copy()

print (df.head())
print (df.columns)
print (df[['Ticker']])
print (df.info())
print (df.describe())
print (df.dtypes)
print (df.isna().sum())
print (df.index.to_series().min())
print (df.index.to_series().max())
print ('Row', is_multi_index)
print ('col', is_multi_col)

           Ticker      Close       High        Low       Open     Volume
Date                                                                    
2019-01-02   AAPL  37.617844  37.839379  36.738855  36.896073  148158800
2019-01-03   AAPL  33.870834  34.711709  33.825574  34.297226  365248800
2019-01-04   AAPL  35.316757  35.385840  34.254350  34.428241  234428400
2019-01-07   AAPL  35.238152  35.452541  34.754589  35.421573  219111200
2019-01-08   AAPL  35.909897  36.164781  35.378693  35.626428  164101200
Index(['Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
           Ticker
Date             
2019-01-02   AAPL
2019-01-03   AAPL
2019-01-04   AAPL
2019-01-07   AAPL
2019-01-08   AAPL
...           ...
2024-12-24   MSFT
2024-12-26   MSFT
2024-12-27   MSFT
2024-12-30   MSFT
2024-12-31   MSFT

[3020 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3020 entries, 2019-01-02 to 2024-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  D

In [6]:
df['LogReturn'] = df.groupby('Ticker')['Close'].transform(lambda x: np.log(x / x.shift(1)))

df.dropna(subset=['LogReturn'], inplace=True)

df['LogReturn_Z'] = df.groupby('Ticker')['LogReturn'].transform(lambda x: (x - x.mean())/ x.std())

print (df.head())
print (df.isna().sum())
print (df[df.isna().any(axis=1)])
print (df.tail())


           Ticker      Close       High        Low       Open     Volume  \
Date                                                                       
2019-01-03   AAPL  33.870834  34.711709  33.825574  34.297226  365248800   
2019-01-04   AAPL  35.316757  35.385840  34.254350  34.428241  234428400   
2019-01-07   AAPL  35.238152  35.452541  34.754589  35.421573  219111200   
2019-01-08   AAPL  35.909897  36.164781  35.378693  35.626428  164101200   
2019-01-09   AAPL  36.519714  36.810329  35.643110  36.038533  180396400   

            LogReturn  LogReturn_Z  
Date                                
2019-01-03  -0.104924    -5.465055  
2019-01-04   0.041803     2.087049  
2019-01-07  -0.002228    -0.179263  
2019-01-08   0.018884     0.907367  
2019-01-09   0.016839     0.802147  
Ticker         0
Close          0
High           0
Low            0
Open           0
Volume         0
LogReturn      0
LogReturn_Z    0
dtype: int64
Empty DataFrame
Columns: [Ticker, Close, High, Low, Open, V

In [7]:

# df.to_csv('/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_daily.CSV')


In [8]:
addres = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_daily.CSV'
raw = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')

def shrinking_ints (df):
    mapping = {}
    for col in df.dtypes [df.dtypes=='int64[pyarrow]'].index:
        max_ = df[col].max()
        min_ = df[col].min()
        if min_ < 0:
            continue
        elif max_ < 255:
            mapping[col] = 'uint8[pyarrow]'
        elif max_ <65_535:
            mapping[col] = 'uint16[pyarrow]'
        elif max_ < 4294967295:
            mapping[col] = 'uint32[pyarrow]'
    return df.astype(mapping)

def clean(df):
    return (df
    .assign(**df.select_dtypes('string').replace('', 'Missing').astype('category'))
    .pipe(shrinking_ints)
    )
    
df = clean(raw)



In [9]:
feat_to_pivot = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']

pivoted_featu = []

for feature in feat_to_pivot:
    feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
    feat_pivot.columns = [f"{ticker}_{feature}" for ticker in feat_pivot.columns]
    pivoted_featu.append(feat_pivot)
    
df_wide = pd.concat(pivoted_featu, axis=1)

org_rows = len(df_wide)
df_wide.dropna(inplace=True)
rows_after_drop = len(df_wide)
if org_rows != rows_after_drop:
    print(1)
    
df_wide = df_wide.sort_index(axis=1)

print (df_wide.head())
print (df_wide.info())
print (df_wide.columns)

/tmp/ipykernel_75403/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_75403/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_75403/607006344.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  feat_pivot = df.pivot_table(index='Date', columns='Ticker', values = feature)
/tmp/ipykernel_75403/6070

            AAPL_Close  AAPL_High  AAPL_LogReturn  AAPL_LogReturn_Z  \
Date                                                                  
2019-01-03   33.870834  34.711709       -0.104924         -5.465055   
2019-01-04   35.316757   35.38584        0.041803          2.087049   
2019-01-07   35.238152  35.452541       -0.002228         -0.179263   
2019-01-08   35.909897  36.164781        0.018884          0.907367   
2019-01-09   36.519714  36.810329        0.016839          0.802147   

             AAPL_Low  AAPL_Open  AAPL_Volume  MSFT_Close  MSFT_High  \
Date                                                                   
2019-01-03  33.825574  34.297226  365248800.0   91.452667  94.072308   
2019-01-04   34.25435  34.428241  234428400.0   95.706047  96.250633   
2019-01-07  34.754589  35.421573  219111200.0    95.82811  96.964225   
2019-01-08  35.378693  35.626428  164101200.0   96.522919  97.621475   
2019-01-09   35.64311  36.038533  180396400.0   97.903183  98.475936  

In [10]:
# output = "/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV"
# df_wide.to_csv(output)

In [14]:
addres = '/home/micheal/Documents/Python_Library/RL_Optimization_Portfolio/data/processed/AP_MS_WIDE_daily.CSV'
df = pd.read_csv(addres, parse_dates=['Date'], index_col='Date', engine='pyarrow', dtype_backend='pyarrow')

features_list = ['Close', 'High', 'Low', 'Open', 'Volume', 'LogReturn', 'LogReturn_Z']

df


,AAPL_Close,AAPL_High,AAPL_LogReturn,AAPL_LogReturn_Z,AAPL_Low,AAPL_Open,AAPL_Volume,MSFT_Close,MSFT_High,MSFT_LogReturn,MSFT_LogReturn_Z,MSFT_Low,MSFT_Open,MSFT_Volume
Date,,,,,,,,,,,,,,
2019-01-03,33.870834,34.711709,-0.104924,-5.465055,33.825574,34.297226,365248800.0,91.452667,94.072308,-0.037482,-2.104201,91.264875,93.9878,42579100.0
2019-01-04,35.316757,35.38584,0.041803,2.087049,34.25435,34.428241,234428400.0,95.706047,96.250633,0.04546,2.432834,92.88923,93.630993,44060600.0
2019-01-07,35.238152,35.452541,-0.002228,-0.179263,34.754589,35.421573,219111200.0,95.82811,96.964225,0.001275,0.015829,94.814061,95.433757,35656100.0
2019-01-08,35.909897,36.164781,0.018884,0.907367,35.378693,35.626428,164101200.0,96.522919,97.621475,0.007224,0.341294,95.499472,96.748262,31514400.0
2019-01-09,36.519714,36.810329,0.016839,0.802147,35.64311,36.038533,180396400.0,97.903183,98.475936,0.014199,0.722791,96.936077,97.518222,32280800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,257.578674,257.58863,0.011413,0.522839,254.675658,254.875189,23234700.0,437.647369,437.916355,0.00933,0.456481,432.527071,432.985301,7164500.0
2024-12-26,258.396667,259.474086,0.003171,0.098619,257.010028,257.568678,27237100.0,436.432068,439.251246,-0.002781,-0.206004,434.957756,437.398354,8194200.0
2024-12-27,254.97493,258.077462,-0.013331,-0.750709,252.451019,257.20953,42355300.0,428.881104,433.553144,-0.017453,-1.008597,424.71712,432.935523,18117700.0


In [ ]:
env = Mult_Asset_portEnv(df=df,
                         num_assets=num_assets,
                         features_list=features_list,
                         window_size=5)

state = env.reset()

for _ in range(10):
    action = env.action_space.sample()
    action = action / np.sum(action)
    next_step, reward, done, info = env.step(action)
    env.render()
    
    if done:
        print('Episode finished.')
        break

NameError: name 'num_assets' is not defined

In [ ]:
env = DummyVecEnv([lambda: PortfolioEnv(df, window_size=5)])

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

/home/micheal/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
------------------------------------
| time/                 |          |
|    fps                | 105      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | -0.0526  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.0368  |
|    value_loss         | 0.00148  |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 120      |
|    iterations         | 200      |
|    time_elapsed       | 8        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.08    |
|    explained_variance | -29.2    |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 0.0547   |
|    value_loss      